In [1]:
from keras.models import Sequential

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf

In [4]:
!pip install keras

In [5]:
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
#from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from PIL import Image
import theano

In [6]:
CATEGORIES = ["DEPRESSED","HEALTHY"]

In [7]:
training = []
def createTrainingData(category):  
  class_num = CATEGORIES.index(category)
  if class_num==0:
    for i in range(101,141):
      j=0
      while True:
        try:
          filename=r"/content/drive/MyDrive/Depression_Detection/Slicing_Experiment/15_sec_and_5_overlap/Spectrograms_Depressed/Mel_AI_"+str(i)+"_result_slice_"+str(j)+".png"
          img_array = cv2.imread(filename)
          new_array = cv2.resize(img_array, (430, 280))
          training.append([new_array, class_num])
          j+=15000
        except:
          break
  elif class_num==1:
    for i in range(301,345):
      j=0
      while True:
        try:
          filename=r"/content/drive/MyDrive/Depression_Detection/Slicing_Experiment/15_sec_and_5_overlap/Spectrograms_Healthy/Mel_AIC_"+str(i)+"_result_slice_"+str(j)+".png"
          img_array = cv2.imread(filename)
          new_array = cv2.resize(img_array, (430, 280))
          training.append([new_array, class_num])
          j+=15000
        except:
          break
  else:
    print("Incorrect input")
createTrainingData("DEPRESSED")

In [8]:
len(training)

2866

In [9]:
createTrainingData("HEALTHY")

In [10]:
len(training)

5815

In [11]:
random.shuffle(training)

In [12]:
X =[]
y =[]
for features, label in training:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, 430,280, 3)

In [ ]:
X = X.astype('float32')
X /= 255
from keras.utils import np_utils
Y = np_utils.to_categorical(y, 2)
Y=np.array(Y)


In [ ]:
print(Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 4)

In [6]:
class myModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1=tf.keras.layers.Conv2D(32,(3,3),padding='same',activation=tf.nn.leaky_relu,input_shape=(430,280,3))
    self.maxpool1=tf.keras.layers.MaxPooling2D((2, 2), strides=2)
    self.conv2=tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.leaky_relu)
    self.maxpool2=tf.keras.layers.MaxPooling2D((2, 2), strides=2)
    self.dropout1=tf.keras.layers.Dropout(0.25)
    self.flatten=tf.keras.layers.Flatten()
    self.dense1=tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu)
    self.dense2=tf.keras.layers.Dense(1,  activation=tf.nn.sigmoid)
  
  def call(self,input):
    conv1_out=self.conv1(input)
    maxpool1_out=self.maxpool1(conv1_out)
    conv2_out=self.conv2(maxpool1_out)
    maxpool2_out=self.maxpool2(conv2_out)
    dropout1_out=self.dropout1(maxpool2_out)
    flatten_out=self.flatten(dropout1_out)
    dense1_out=self.dense1(flatten_out)
    dense2_out=self.dense2(dense1_out)
    return dense2_out

In [7]:
import sys,os,random
train_file=r"/content/drive/MyDrive/Depression_Detection/Slicing_Experiment/15_sec_and_5_overlap/filenames_train.txt"
all_data=[]
f=open(train_file)
line = f.readline()
while line:
  info = line.strip("\n").split(",")
  filename = info[0]
  label = int(info[-1])
  all_data.append([filename,label])
  # print(filename,label)
  line = f.readline()
  # break

N = len(all_data)
print("Total %d data"%N)

Total 5815 data


In [8]:
split = 0.2
n_train = 1-int(N*0.2)
random.shuffle(all_data)
train_data = all_data[:n_train]
test_data = all_data[n_train:]
print("%d Train data %d Test data"%(len(train_data),len(test_data)))

4653 Train data 1162 Test data


In [9]:
def generate_batch(data,batch_size):
  total = len(data)
  start = 0
  while start < total:
    end = min(total,start + batch_size)
    batch_data = data[start:end]
    batch_x = []
    batch_y = []
    for filename,label in batch_data:
      try:
        img_array = cv2.imread(filename)
        new_array = cv2.resize(img_array, (430, 280))
        batch_x.append(new_array)
        one_hot = [0.0,0.0]
        one_hot[label] = 1.0
        batch_y.append(one_hot)
      except:
        pass
    # print(feat.shape,one_hot)
    batch_x = np.array(batch_x).reshape(-1, 430,280, 3)
    batch_x = batch_x.astype('float32')
    batch_x/= 255
    yield batch_x,np.array(batch_y),start,end
    start = end
  

batch=generate_batch(train_data,32)

In [10]:
p=next(batch)

In [11]:
p[0].shape

(32, 430, 280, 3)

In [12]:
model_2D = myModel()
initial_learning_rate = 0.0000001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=18620,
    decay_rate=0.5,
    staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)


In [ ]:
epochs = 250
batch_size = 5

N = len(train_data)
n_batches = int(np.ceil(N/batch_size))
print("%d total batches"%n_batches)
losses={}
accuracies={}

for e in range(epochs):
  batch = generate_batch(train_data,batch_size)
  average_accuracy=0
  avg_loss = 0
  for b in range(n_batches):
    # get data for a batch
    try:
      batch_x,batch_y,s,end = next(batch)
      # print("Batch IDX %d -  start %d end %d"%(b,s,e))
      # backprop
      with tf.GradientTape() as tape:
        output = model_2D(batch_x)
        loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(output,batch_y))
        avg_loss += loss
        weights = model_2D.trainable_variables
        grad = tape.gradient(loss,weights)
        optimizer.apply_gradients(zip(grad,weights))
      #to compute accuracy
        pl = tf.argmax(output)
        tl = tf.argmax(batch_y)
        accuracy=tf.reduce_mean(tf.cast(tf.equal(pl,tl),tf.float32))
        average_accuracy+=accuracy.numpy()
    except:
      break
  average_accuracy/=n_batches
  print("Total loss in epoch "+str(e)+ " is "+ str(avg_loss.numpy())+" Accuracy is " + str(average_accuracy))
  accuracies[e+1]= average_accuracy
  losses[e+1]=avg_loss.numpy()
epochs=[]
for item in losses.keys():
  epochs.append(item)
import pandas as pd
loss_values=[]
for i in range(len(epochs)):
  loss_values.append(losses[i+1])
accuracy_values=[]
for i in range(len(epochs)):
  accuracy_values.append(accuracies[i+1])
dictionary={'Epoch Number': epochs, 'Accuracy': accuracy_values, 'Loss': loss_values}
data=pd.DataFrame.from_dict(dictionary)
import matplotlib.pyplot as plt
%matplotlib inline
data.plot(x='Epoch Number', y='Loss', kind='line', ylabel='Loss')
data.plot(x='Epoch Number', y='Accuracy', kind='line', ylabel='Loss')

931 total batches
Total loss in epoch 0 is 7137.308223978188 Accuracy is 0.2577873254564984
Total loss in epoch 1 is 7033.391239462199 Accuracy is 0.3104189044038668
Total loss in epoch 2 is 6935.535790562902 Accuracy is 0.3109559613319012
Total loss in epoch 3 is 6826.848167297698 Accuracy is 0.31954887218045114
Total loss in epoch 4 is 6709.338310621953 Accuracy is 0.3276047261009667
Total loss in epoch 5 is 6605.788117336313 Accuracy is 0.3281417830290011
Total loss in epoch 6 is 6486.537478348953 Accuracy is 0.3335123523093448
Total loss in epoch 7 is 6375.987409379712 Accuracy is 0.33404940923737914
Total loss in epoch 8 is 6255.3120615432035 Accuracy is 0.33297529538131043
Total loss in epoch 9 is 6131.150259783432 Accuracy is 0.3351235230934479
Total loss in epoch 10 is 6020.069722154993 Accuracy is 0.3372717508055854
Total loss in epoch 11 is 5921.672794336122 Accuracy is 0.3372717508055854
Total loss in epoch 12 is 5833.916387581491 Accuracy is 0.3383458646616541
Total loss in

In [ ]:
model2D.save(r"/content/drive/MyDrive/Depression_Detection/Slicing_Experiment/15_sec_and_5_overlap/CNN20Model")

In [ ]:
#Exploratory Model
'''
model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),padding='same',activation=tf.nn.leaky_relu,input_shape=(430,280,3)),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.leaky_relu),
        tf.keras.layers.MaxPooling2D((2, 2), strides=2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(2,  activation=tf.nn.sigmoid)
])
'''

In [ ]:
#Resnet
"""
Resnet code
"""

In [ ]:

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
history=model.fit(X_train, y_train, batch_size =5, epochs =50, verbose = 1, validation_data = (X_test, y_test))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()